In [1]:
import os, sys
from os.path import join

import nibabel as nib
import numpy as np
import pandas as pd

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Make timing files for each run

   - each run was composed of 6 blocks. Each block began with 20s of rest, and then 10s of task (either motor squeezing or imagery)
   - there was 6 seconds of disdaqs at the beginning
   - However, the data collection started 2 TRs AFTER the task started. Thus with a TR of 1s, there were 2 seconds of task before the task started. Likewise, for a TR of 2s, there was 4 seconds of task before the task started. This was due to the 2 dummy scans that Philips inserts at the beginning of the each scan

In [33]:
trialStarts = []
currentTime = 0

for b in range(6):
    # start time for rest
    trialStarts.append(currentTime)

    # increment by rest duration
    currentTime += 20
    
    # start time for task
    trialStarts.append(currentTime)
    
    # increment by the task duration
    currentTime += 10

# add in offset for 
trialStarts = np.array(trialStarts) + 6

In [35]:
## make separate arrays for TR=1s and TR=2s
# subtract 2 dummy scans (2s)
onsets_1sTR = trialStarts - 2

# subtract 2 dummy scans (4s)
onsets_2sTR = trialStarts - 4

In [5]:
# hardcoded order on each run
trialTypes = ['rest', 'motor', 
             'rest', 'imagery',
             'rest', 'imagery',
             'rest', 'motor',
             'rest', 'imagery',
             'rest', 'motor']


In [46]:
# make a dataframe
trialOnsets = pd.DataFrame({'trialTypes': trialTypes, 'onsets_TR1': onsets_1sTR, 'onsets_TR2': onsets_2sTR, 'weights': np.ones(onsets_1sTR.shape)})

In [52]:
def trialDuration(row):
    if row['trialTypes'] == 'rest':
        val = 20
    else:
        val = 10
    return val
trialOnsets['duration'] = trialOnsets.apply(trialDuration, axis=1)

In [53]:
trialOnsets

,onsets_TR1,onsets_TR2,trialTypes,weights,duration
0,4,2,rest,1.0,20
1,24,22,motor,1.0,10
2,34,32,rest,1.0,20
3,54,52,imagery,1.0,10
4,64,62,rest,1.0,20
5,84,82,imagery,1.0,10
6,94,92,rest,1.0,20
7,114,112,motor,1.0,10
8,124,122,rest,1.0,20
9,144,142,imagery,1.0,10


### Write out FSL 3col files

In [63]:
# 1s TR files
for trialType in ['rest', 'motor', 'imagery']:
    df = trialOnsets.loc[trialOnsets.trialTypes == trialType, ['onsets_TR1','duration', 'weights']]
    df.to_csv('../data/subject001/timingFiles/TR1_{}.txt'.format(trialType),
              sep='\t', 
              header=False,
              index=False)
    
# 2s TR files
for trialType in ['rest', 'motor', 'imagery']:
    df = trialOnsets.loc[trialOnsets.trialTypes == trialType, ['onsets_TR2','duration', 'weights']]
    df.to_csv('../data/subject001/timingFiles/TR2_{}.txt'.format(trialType),
              sep='\t', 
              header=False,
              index=False)

In [49]:
# write out FSL 3col timing file
trialOnsets[['onsets_TR1','duration', 'weights']]

,trialTypes,weights
0,rest,1.0
1,motor,1.0
2,rest,1.0
3,imagery,1.0
4,rest,1.0
5,imagery,1.0
6,rest,1.0
7,motor,1.0
8,rest,1.0
9,imagery,1.0
